In [70]:
import numpy as np
import pandas as pd
import os
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.metrics import auc, roc_curve, roc_auc_score, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

# Orbital (left)

In [58]:
crops_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/ORBITAL/mask'
label='Left_OFC'
side = 'L'
labels = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_OFC_labels_from_0.csv', usecols=['Subject', label])
splits_dir = '/neurospin/dico/data/deep_folding/current/datasets/orbital_patterns/Troiani/Left'

# FIP (right)

In [64]:
crops_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.I.P./mask'
label='Right_FIP'
side = 'R'
labels = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/FIP_labels.csv', usecols=['Subject', label])
splits_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/FIP/Right' ## NB : CHANGE FOR LEFT

# Isomap CS (left)

In [ ]:
crops_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.C.-sylv./mask'
label='Isomap_central_left_dim1' ## loop on dimensions !!
side = 'L'
labels = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_isomap_labels_SC-sylv_left.csv', usecols=['Subject', label])
splits_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/Isomap/splits' ## NB : CHANGE FOR LEFT

load data

In [65]:
skels = np.load(os.path.join(crops_dir, f'{side}skeleton.npy'))
skel_subs = pd.read_csv(os.path.join(crops_dir, f'{side}skeleton_subject.csv'))
skels = skels.reshape(skels.shape[0], np.prod(skels.shape[1:]))
skels = skels.astype(bool)

train_subs = pd.read_csv(os.path.join(splits_dir, 'train_split.csv'), names=['Subject'])
val_subs = pd.read_csv(os.path.join(splits_dir, 'val_split.csv'), names=['Subject'])
test_subs = pd.read_csv(os.path.join(splits_dir, 'test_split.csv'), names=['Subject'])

proportions = np.unique(labels[label], return_counts=True)
proportions = proportions[1] / np.sum(proportions[1])

train = skel_subs.loc[skel_subs['Subject'].isin(train_subs['Subject'])]
idxs_train = train.index.tolist()
Y_train = pd.merge(train, labels)[label].to_numpy().reshape(-1,1)
train_skels= skels[idxs_train]

val = skel_subs.loc[skel_subs['Subject'].isin(val_subs['Subject'])]
idxs_val = val.index.tolist()
Y_val = pd.merge(val, labels)[label].to_numpy().reshape(-1,1)
val_skels= skels[idxs_val]

test = skel_subs.loc[skel_subs['Subject'].isin(test_subs['Subject'])]
idxs_test = test.index.tolist()
Y_test = pd.merge(test, labels)[label].to_numpy().reshape(-1,1)
test_skels= skels[idxs_test]

define folds for gridsearch

In [71]:
# define train val and their indexes for gridsearch cv
X_train_val = np.vstack((train_skels, val_skels))
Y_train_val = np.vstack((Y_train, Y_val))
split_index = np.full(len(X_train_val), -1)  # All initially train
split_index[len(train_skels):] = 0  # Mark validation samples
predefined_split = PredefinedSplit(test_fold=split_index)

# Classification

In [20]:
# logistic
parameters={'l1_ratio': np.linspace(0,1,11), 'C': [10**k for k in range(-3,3)]}
model = LogisticRegression(solver='saga', penalty='elasticnet',
                           max_iter=2000, random_state=0)

grid_search = GridSearchCV(model, parameters, cv=predefined_split, scoring='roc_auc_ovr_weighted', refit=True)
grid_search.fit(X_train_val, Y_train_val)
# Print best parameters
print("Best parameters:", grid_search.best_params_)
# Evaluate on the test set
best_model = grid_search.best_estimator_

/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STS

Best parameters: {'C': 1, 'l1_ratio': np.float64(1.0)}


/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [21]:
y_test_pred = best_model.predict_proba(test_skels)
#score = roc_auc_score(Y_test, y_test_pred, multi_class='ovr', average='weighted') ## WON'T WORK WITH SINGLE CLASS
#print(score)
score = roc_auc_score(Y_test, y_test_pred[:, 1])
print(score)

1.0


In [67]:
# rbf svm ## penalty is l2
parameters={'C': [10**k for k in range(-3,3)]}
model = SVC(kernel='rbf', probability=True,
            max_iter=2000, random_state=0, decision_function_shape='ovr')

predefined_split = PredefinedSplit(test_fold=split_index)

grid_search = GridSearchCV(model, parameters, cv=predefined_split, scoring='roc_auc_ovr_weighted', refit=True)
grid_search.fit(X_train_val, Y_train_val)
# Print best parameters
print("Best parameters:", grid_search.best_params_)
# Evaluate on the test set
best_model = grid_search.best_estimator_

/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/neurospin/dico/jlaval/Runs_jl277509/2023_jlaval_STS

Best parameters: {'C': 10}


In [69]:
y_test_pred = best_model.predict_proba(test_skels)
#score = roc_auc_score(Y_test, y_test_pred, multi_class='ovr', average='weighted')
#print(score)
score = roc_auc_score(Y_test, y_test_pred[:, 1])
print(score)

0.6304012345679012


# Regression

In [ ]:
# elastic net

parameters={'l1_ratio': np.linspace(0,1,11), 'alpha': [10**k for k in range(-4,3)]}   
model = ElasticNet()

predefined_split = PredefinedSplit(test_fold=split_index)

grid_search = GridSearchCV(model, parameters, cv=predefined_split, scoring='r2', refit=True)
grid_search.fit(X_train_val, Y_train_val)
# Print best parameters
print("Best parameters:", grid_search.best_params_)
# Evaluate on the test set
best_model = grid_search.best_estimator_

In [ ]:
y_test_pred = best_model.predict_proba(test_skels)
score = r2_score(Y_test, y_test_pred, multi_class='ovr', average='weighted')
print(score)

In [ ]:
# SVR ## penalty is l2
parameters={'C': [10**k for k in range(-3,3)], 'gamma': ['scale', 'auto']}   
model = SVR(kernel='rbf')

predefined_split = PredefinedSplit(test_fold=split_index)

grid_search = GridSearchCV(model, parameters, cv=predefined_split, scoring='r2', refit=True)
grid_search.fit(X_train_val, Y_train_val)
# Print best parameters
print("Best parameters:", grid_search.best_params_)
# Evaluate on the test set
best_model = grid_search.best_estimator_

In [ ]:
y_test_pred = best_model.predict_proba(test_skels)
score = r2_score(Y_test, y_test_pred, multi_class='ovr', average='weighted')
print(score)